# Image Transformations
*Author: Vladislav Kim*
* [Introduction](#intro)
* [Denoising](#noise)
* [Thresholding: separate foreground from background](#threshold)
* [Non-uniform illumination correction](#bgcorrect)
* [Morphological operations](#morphology)

## Introduction
<a id="intro"></a> 
Once we can handle images of any OME-supported format in Python, we can apply a number of transformations to correct brightness, contrast, noise levels, etc


After the images have been loaded and inspected, they can be transformed. To name a few image transformations:
+ adjust brightness and contrast
+ rescale image
+ remove background
+ denoise / introduce Gaussian blur
+ separate foreground from background (threshold)

Here we will talk about the most basic image transformations such as contrast adjustment and rescaling (see 'Advanced Image Processing' notebook for the detailed treatment of image transformations).


Perhaps the most trivial image transformation is image inversion (obtaining the "negative" of the image)

To adjust brightness of an image, one may have to apply gamma correction to the image, which simply raises the `
np.array` to the power of `gamma`

A useful feature is image rescaling, which can be achieved by `skimage.transform.resize`:

The best is always to compare the original and transformed image side-by-side:

## Denoising
<a id="noise"></a> 
Smoothing for speckle noise. Talk about Fourier-space analysis for complex noise structure and non-local means denoising.

## Thresholding: separate foreground from background
<a id="threshold"></a> 

## Illumination correction
<a id="bgcorrect"></a>
Non-uniform background removal

## Morphological operations
<a id="morphology"></a> 